# Region ID Prediction - SMAI Mini Project

In [ ]:
# import torch
# import gc
# from google.colab import runtime

# def clean_gpu():
#     # Clear PyTorch cache
#     torch.cuda.empty_cache()

#     # Force Python garbage collection
#     gc.collect()

#     # Colab-specific memory release
#     try:
#         runtime.unassign()
#     except:
#         pass

#     # NVIDIA GPU reset (if available)
#     if torch.cuda.is_available():
#         torch.cuda.reset_peak_memory_stats()
#         torch.cuda.reset_accumulated_memory_stats()
#         torch.cuda.ipc_collect()

#     # Verify free memory
#     free, total = torch.cuda.mem_get_info()
#     print(f"✅ GPU Memory Cleared | Free: {free/1024**3:.2f}GB/{total/1024**3:.2f}GB")

# clean_gpu()  # Execute this before anything else


In [2]:
# !pip install efficientnet-pytorch torch-optimizer

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 929.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.4 MB/s eta 0:00:00
 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import gdown
# gdown.download_folder("https://drive.google.com/drive/folders/1vtLNqk0N2GriYKcv6FUwiPQHSCBN69E-", quiet=False, use_cookies=False)


# import zipfile
# with zipfile.ZipFile("/content/Phase_2_data/images_val.zip", 'r') as zip_ref:
#     zip_ref.extractall("dataset")
# with zipfile.ZipFile("/content/Phase_2_data/images_train.zip", 'r') as zip_ref:
#     zip_ref.extractall("dataset")




Mounted at /content/drive


Retrieving folder contents


Processing file 1hQrS-2OIDf5ceoSZte48OzNn_znS1VQT images_train.zip
Processing file 1aHLzBTGzGeuEoNusam0WZPk9XKitClTV images_val.zip
Processing file 1df0A-MN5h1KW_j3B9RqeRvHWWp_iecpg labels_train.csv
Processing file 1en9Vq8sM94l09WMyxurAqVzV1VG5UlHV labels_val.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1hQrS-2OIDf5ceoSZte48OzNn_znS1VQT
From (redirected): https://drive.google.com/uc?id=1hQrS-2OIDf5ceoSZte48OzNn_znS1VQT&confirm=t&uuid=b5f7d710-3f3c-4ea2-8a6d-89c6b9de5b6c
To: /content/Phase_2_data/images_train.zip
100%|██████████| 616M/616M [00:04<00:00, 131MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1aHLzBTGzGeuEoNusam0WZPk9XKitClTV
From (redirected): https://drive.google.com/uc?id=1aHLzBTGzGeuEoNusam0WZPk9XKitClTV&confirm=t&uuid=9309bbb0-4d80-47c0-95d8-f0e92723f3a3
To: /content/Phase_2_data/images_val.zip
100%|██████████| 36.2M/36.2M [00:00<00:00, 193MB/s]
Downloading...
From: https://drive.google.com/uc?id=1df0A-MN5h1KW_j3B9RqeRvHWWp_iecpg
To: /content/Phase_2_data/labels_train.csv
100%|██████████| 257k/257k [00:00<00:00, 96.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1en9Vq8sM94

In [3]:
# Install required packages
# !pip install efficientnet-pytorch torch-optimizer

import os
import math
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import warnings
from torch.cuda.amp import GradScaler, autocast

# Suppress warnings
warnings.filterwarnings("ignore")
NUM_WORKERS = min(4, os.cpu_count() // 2)

# ===== Hyperparameters =====
# Training Parameters
EPOCHS = 25
BATCH_SIZE = 28
WARMUP_EPOCHS = 5
PATIENCE = 10  # Early stopping patience
GRADIENT_ACCUMULATION_STEPS = 2

# Optimization Parameters
INIT_LR = 5e-4
MIN_LR = 9e-5
WEIGHT_DECAY = 1e-4
MAX_GRAD_NORM = 1.0  # Gradient clipping

# Model Architecture
NUM_FEATURES = 512
ATTENTION_DIM = 1024
DROPOUT_RATE = 0.5
LABEL_SMOOTHING = 0.1
UNFREEZE_LAST_N_BLOCKS = 15  # For EfficientNet

# Data Augmentation
IMAGE_SIZE = 352  # Final crop size
INITIAL_RESIZE = 384  # Initial resize before augmentations

# AMP Settings
if torch.__version__ >= '2.0':
    torch.set_float32_matmul_precision('high')

scaler = GradScaler()
amp_dtype = torch.float16

# ===== Data Loading =====
train_df = pd.read_csv("/content/Phase_2_data/labels_train.csv")
val_df = pd.read_csv("/content/Phase_2_data/labels_val.csv")

# Preprocessing
for df in [train_df, val_df]:
    df['hour'] = df['timestamp'].str.split(":").str[0].astype(int)
    df['Region_ID'] = df['Region_ID'].astype(int)
    df['label'] = df['Region_ID'] - 1  # 0-indexed

minmax_scaler = MinMaxScaler()
train_tabular = train_df[['latitude', 'longitude', 'angle', 'hour']]
val_tabular = val_df[['latitude', 'longitude', 'angle', 'hour']]

minmax_scaler.fit(train_tabular)
train_df[['latitude', 'longitude', 'angle', 'hour']] = minmax_scaler.transform(train_tabular)
val_df[['latitude', 'longitude', 'angle', 'hour']] = minmax_scaler.transform(val_tabular)

# ===== Transforms =====
train_transform = transforms.Compose([
    transforms.Resize((INITIAL_RESIZE, INITIAL_RESIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomAffine(degrees=30, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.7, 1.0), ratio=(0.8, 1.2)),
    transforms.ToTensor(),  # Convert to tensor before tensor-based transforms
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.2)),  # Now works with tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((INITIAL_RESIZE, INITIAL_RESIZE)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ===== Dataset =====
class MultimodalDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Tabular data
        tabular = torch.tensor([
            float(row['latitude']),
            float(row['longitude']),
            float(row['angle']),
            float(row['hour'])
        ], dtype=torch.float32)

        label = torch.tensor(row['label'], dtype=torch.long)

        return image, tabular, label

# ===== Model =====
class EfficientNetMultimodal(nn.Module):
    def __init__(self, num_classes=15):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b5')
        self.img_feature_dim = 2048

        # Freeze/unfreeze layers
        for param in self.efficientnet.parameters():
            param.requires_grad = False
        for param in self.efficientnet._blocks[-UNFREEZE_LAST_N_BLOCKS:].parameters():
            param.requires_grad = True

        self.tabular_net = nn.Sequential(
            nn.Linear(4, NUM_FEATURES),
            nn.BatchNorm1d(NUM_FEATURES),
            nn.GELU(),
            nn.Dropout(DROPOUT_RATE/2),
            nn.Linear(NUM_FEATURES, NUM_FEATURES),
            nn.BatchNorm1d(NUM_FEATURES),
            nn.GELU(),
            nn.Dropout(DROPOUT_RATE/2),
        )

        self.attention = nn.Sequential(
            nn.Linear(2048 + NUM_FEATURES, ATTENTION_DIM),
            nn.GELU(),
            nn.Dropout(DROPOUT_RATE),
            nn.Linear(ATTENTION_DIM, 1),
            nn.Sigmoid()
        )

        self.classifier = nn.Sequential(
            nn.Linear(2048 + NUM_FEATURES, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(DROPOUT_RATE),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, image, tabular):
        img_feat = self.efficientnet.extract_features(image)
        img_feat = F.adaptive_avg_pool2d(img_feat, 1).flatten(1)
        tab_feat = self.tabular_net(tabular)
        combined = torch.cat([img_feat, tab_feat], dim=1)
        attn_weights = self.attention(combined)
        attended = combined * attn_weights
        return self.classifier(attended)

# ===== DataLoaders =====
train_dataset = MultimodalDataset(train_df, "/content/dataset/images_train", train_transform)
val_dataset = MultimodalDataset(val_df, "/content/dataset/images_val", val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True if NUM_WORKERS > 0 else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True if NUM_WORKERS > 0 else False
)

# ===== Training Function =====
def train_model(model, train_loader, val_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=INIT_LR, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)

    # Learning rate scheduler with warmup
    def lr_lambda(epoch):
        if epoch < WARMUP_EPOCHS:
            return (epoch + 1) / WARMUP_EPOCHS
        else:
            progress = (epoch - WARMUP_EPOCHS) / (EPOCHS - WARMUP_EPOCHS)
            return MIN_LR/INIT_LR + 0.5*(1 - MIN_LR/INIT_LR)*(1 + math.cos(math.pi*progress))

    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    best_acc = 0.0
    no_improve = 0
    history = {'train_loss': [], 'train_acc': [], 'val_acc': [], 'lr': []}

    for epoch in range(EPOCHS):
        model.train()
        total_loss, total_correct = 0, 0
        optimizer.zero_grad()

                # TEST MODE: Break after 5 batches
        TEST_MODE = False
        batch_counter = 0

        for i, (images, tabular, labels) in enumerate(train_loader):
            images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)

            with autocast(dtype=amp_dtype):
                outputs = model(images, tabular)
                loss = criterion(outputs, labels) / GRADIENT_ACCUMULATION_STEPS

            scaler.scale(loss).backward()

            if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                if MAX_GRAD_NORM > 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            total_loss += loss.item() * images.size(0) * GRADIENT_ACCUMULATION_STEPS
            total_correct += (outputs.argmax(1) == labels).sum().item()

            # TEST MODE: Break after 5 batches
            # batch_counter += 1
            # if TEST_MODE and batch_counter >= 5:
            #     print("\nTEST MODE: Breaking after 5 batches")
            #     break

        # Skip validation in test mode
        # if TEST_MODE:
            # Save dummy model for prediction testing
            # torch.save(model.state_dict(), 'test_model.pth')
            # print("Saved test model after 5 batches")
            # return None, None  # Return early for testing

        # Validation
        model.eval()
        val_correct = 0
        with torch.no_grad():
            for images, tabular, labels in val_loader:
                images, tabular, labels = images.to(device), tabular.to(device), labels.to(device)
                with autocast(dtype=amp_dtype):
                    outputs = model(images, tabular)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        # Calculate metrics
        train_loss = total_loss / len(train_loader.dataset)
        train_acc = total_correct / len(train_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])

        # Learning rate scheduling
        scheduler.step()

        # Early stopping and model checkpoint
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model_RI_combine.pth')
            no_improve = 0
            print(f"New best model saved with val_acc: {best_acc:.4f}")
        else:
            no_improve += 1
            if no_improve >= PATIENCE:
                print(f"Early stopping at epoch {epoch+1}")
                break

        print(f"Epoch {epoch+1}/{EPOCHS} | LR: {history['lr'][-1]:.2e} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Acc: {val_acc:.4f} | Best: {best_acc:.4f}")

    # Save training plot
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.title('Training Loss')
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Acc')
    plt.plot(history['val_acc'], label='Val Acc')
    plt.title('Accuracy')
    plt.legend()
    plt.savefig('training_history.png')
    plt.close()

    return best_acc, history

# ===== Test Dataset and Loader =====
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg')])
        self.tabular_data = torch.zeros((len(self.image_files), 4), dtype=torch.float32)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.tabular_data[idx]

# ===== Prediction Function =====
def predict_with_tta(model, loader, n_aug=5):
    model.eval()
    device = next(model.parameters()).device
    all_preds = []

    with torch.no_grad():
        for batch in loader:
            # Handle both cases: (img, tab) and (img, tab, label)
            if len(batch) == 3:
                images, tabular, _ = batch  # ignore labels for validation
            else:
                images, tabular = batch  # test loader only has 2 elements

            images, tabular = images.to(device), tabular.to(device)
            outputs = torch.zeros((images.size(0), 15), device=device)

            for _ in range(n_aug):
                with autocast(dtype=amp_dtype):
                    aug_imgs = []
                    for img in images:
                        img_np = img.cpu().numpy().transpose(1, 2, 0)
                        img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                        img_np = (img_np * 255).astype('uint8')
                        aug_img = train_transform(Image.fromarray(img_np))
                        aug_imgs.append(aug_img)
                    aug_imgs = torch.stack(aug_imgs).to(device)
                    outputs += model(aug_imgs, tabular)

            all_preds.extend((outputs.argmax(1) + 1).cpu().numpy())
    return all_preds



# def get_test_images_count(test_dir):
#     """Verify the exact number of test images"""
#     return len([f for f in os.listdir(test_dir) if f.endswith('.jpg')])

# ===== Main Execution =====
if __name__ == "__main__":
    # Initialize model
    model = EfficientNetMultimodal(num_classes=15)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Compile if available
    if torch.__version__ >= '2.0':
        try:
            model = torch.compile(model, mode='default', fullgraph=False, dynamic=False)
            print("Model compiled with compatible settings")
        except Exception as e:
            print(f"Compilation failed: {str(e)} - Running uncompiled")

    # Train and select best model based on val accuracy
    best_acc, history = train_model(model, train_loader, val_loader)

    # Only proceed with prediction if not in TEST MODE
    if best_acc is not None:
        # Load best model
        model.load_state_dict(torch.load('best_model_RI_combine.pth'))
        print(f"Loaded best model with val accuracy: {best_acc:.4f}")

        # Verify test image count
        test_dir = "/content/drive/MyDrive/images_test"  # Update this path
        # test_count = get_test_images_count(test_dir)
        # print(f"Found {test_count} test images")

        # if test_count != 369:
        #     print(f"⚠️ Warning: Expected 369 test images, found {test_count}")
        #     print("Listing first 10 files:")
        #     print(os.listdir(test_dir)[:10])

        # Create test dataset with exactly 369 samples
        class FixedTestDataset(Dataset):
            def __init__(self, img_dir, transform=None):
                self.img_dir = img_dir
                self.transform = transform
                # Get all supported image formats (case insensitive)
                self.image_files = sorted([
                    f for f in os.listdir(img_dir)
                    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
                ])[:369]  # Take first 369 images

                # Verify we got enough images
                if len(self.image_files) < 369:
                    print(f"Warning: Only found {len(self.image_files)} images in {img_dir}")
                    # Pad with dummy filenames if needed (will raise error if accessed)
                    self.image_files.extend([None] * (369 - len(self.image_files)))

                self.tabular_data = torch.zeros((369, 4), dtype=torch.float32)

            def __len__(self):
                return 369  # Always return 369

            def __getitem__(self, idx):
                if self.image_files[idx] is None:
                    raise ValueError(f"Missing image at index {idx} - only found {len([f for f in self.image_files if f is not None])} real images")

                img_path = os.path.join(self.img_dir, self.image_files[idx])
                try:
                    image = Image.open(img_path).convert("RGB")
                    if self.transform:
                        image = self.transform(image)
                    return image, self.tabular_data[idx]
                except Exception as e:
                    print(f"Error loading image {img_path}: {str(e)}")
                    # Return a blank image if there's an error loading
                    blank_image = Image.new('RGB', (IMAGE_SIZE, IMAGE_SIZE))
                    if self.transform:
                        blank_image = self.transform(blank_image)
                    return blank_image, self.tabular_data[idx]

        # Create loaders
        test_dataset = FixedTestDataset(test_dir, val_transform)
        test_loader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=NUM_WORKERS
        )

        # Generate predictions
        val_preds = predict_with_tta(model, val_loader)[:369]  # Ensure 369
        test_preds = predict_with_tta(model, test_loader)[:369]  # Ensure 369

        # Create submission
        submission = pd.DataFrame({
            'id': list(range(738)),
            'Region_ID': val_preds + test_preds
        })

        # Verify
        assert len(submission) == 738, f"Expected 738, got {len(submission)}"
        submission.to_csv('2022101002_4_RI_combined_xyz.csv', index=False)
        print("✅ Saved predictions for 369 val + 369 test samples")
    else:
        print("Test mode completed - skipping prediction phase")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth
100%|██████████| 117M/117M [00:00<00:00, 130MB/s]


Loaded pretrained weights for efficientnet-b5
Model compiled with compatible settings


W0504 09:56:52.837000 261 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


New best model saved with val_acc: 0.4065
Epoch 1/25 | LR: 1.00e-04 | Train Loss: 2.4779 | Train Acc: 0.2178 | Val Acc: 0.4065 | Best: 0.4065
New best model saved with val_acc: 0.6314
Epoch 2/25 | LR: 2.00e-04 | Train Loss: 1.8658 | Train Acc: 0.4630 | Val Acc: 0.6314 | Best: 0.6314
New best model saved with val_acc: 0.7534
Epoch 3/25 | LR: 3.00e-04 | Train Loss: 1.5126 | Train Acc: 0.6068 | Val Acc: 0.7534 | Best: 0.7534
New best model saved with val_acc: 0.8076
Epoch 4/25 | LR: 4.00e-04 | Train Loss: 1.3181 | Train Acc: 0.6952 | Val Acc: 0.8076 | Best: 0.8076
Epoch 5/25 | LR: 5.00e-04 | Train Loss: 1.2155 | Train Acc: 0.7435 | Val Acc: 0.7886 | Best: 0.8076
New best model saved with val_acc: 0.8564
Epoch 6/25 | LR: 5.00e-04 | Train Loss: 1.0940 | Train Acc: 0.7923 | Val Acc: 0.8564 | Best: 0.8564
Epoch 7/25 | LR: 4.97e-04 | Train Loss: 1.0244 | Train Acc: 0.8224 | Val Acc: 0.8238 | Best: 0.8564
Epoch 8/25 | LR: 4.90e-04 | Train Loss: 0.9551 | Train Acc: 0.8491 | Val Acc: 0.8482 | Bes